## Model Training

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [2]:
df = pd.read_csv('data/student.csv')

df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


For fairness I will be droping gender & race_ethnicity and see how the model performs 

In [3]:
df_new = df.copy()

df_new = df_new.drop(['gender','race_ethnicity'], axis = 1)
df_new.head()

,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,bachelor's degree,standard,none,72,72,74
1,some college,standard,completed,69,90,88
2,master's degree,standard,none,90,95,93
3,associate's degree,free/reduced,none,47,57,44
4,some college,standard,none,76,78,75


#### Train test split 

In [4]:
X = df_new.drop('math_score', axis = 1)
y = df_new['math_score']

X.shape, y.shape

((1000, 5), (1000,))

In [5]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [6]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 5), (200, 5))

In [7]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

X_train.shape, X_test.shape

((800, 12), (200, 12))

So I have done this to avoid data leakage. 

### Create an Evaluate Function to give all metrics after model Training

In [8]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [9]:
models = {
    "Linear Regression": LinearRegression(n_jobs=-1),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(n_jobs=-1),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(n_jobs=-1),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 8.2220
- Mean Absolute Error: 6.6800
- R2 Score: 0.7001
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 8.5977
- Mean Absolute Error: 7.0047
- R2 Score: 0.6962


Lasso
Model performance for Training set
- Root Mean Squared Error: 8.5163
- Mean Absolute Error: 6.8618
- R2 Score: 0.6783
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 8.7448
- Mean Absolute Error: 7.2325
- R2 Score: 0.6857


Ridge
Model performance for Training set
- Root Mean Squared Error: 8.2038
- Mean Absolute Error: 6.6615
- R2 Score: 0.7015
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 8.5736
- Mean Absolute Error: 6.9816
- R2 Score: 0.6979


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 7.5481
- Mean Absolute Error: 6.0600
- R2 Score: 0.7473
-----------------------

In [13]:
results = pd.DataFrame(list(zip(model_list, r2_list)), 
                       columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],
                                                                       ascending=False)
results

,Model Name,R2_Score
2,Ridge,0.697925
0,Linear Regression,0.696225
8,AdaBoost Regressor,0.691015
1,Lasso,0.685738
7,CatBoosting Regressor,0.642989
5,Random Forest Regressor,0.637358
3,K-Neighbors Regressor,0.581124
6,XGBRegressor,0.564430
4,Decision Tree,0.367121
